## SICP 习题 （3.25）解题总结: 多维表格的处理

SICP 习题 3.25 希望我们实现多维表格。

就像我们讨论一维数组和二维数组一样，当我们学会处理一维表格后就可以处理二维表格了，处理方法不过就是增加一个key而已。那要处理多维表格，就是处理多个key的表格。

这里想强调的是书中实现的表格并不是严格要求的表格，不同行可以拥有不同数量的列。因为这里的表格是通过指针实现的，并不是通过等长数据段实现的。这里的表格有点像我们在普通编程书里面提到的不定长数组。

处理多维表格，关键在与多个key的表格，如果是指定三维或者四维，无外乎就是改代码加多几个参数而已。但是，已经建立抽象能力的我们，怎么会满足于重复相同的代码呢？我们要构建一个能处理不定长key表格的函数。

要处理不定长key，把不定长key表达为一个列表，然后不断递归处理就好了。



In [7]:
(define (assoc key records same-key?)
  (cond ((null? records) #f)
	((same-key? key (caar records)) (car records))
	(else (assoc key (cdr records) same-key?))))

(define (make-table same-key?)
  (let ((local-table (list '*table*)))
    (define (lookup keys)
      (define (inter-lookup keys inter-table same-key?)
	(if (null? keys)
	      (if inter-table
		  (cdr inter-table)
		  (begin
		    (display "record not found ") (display keys) (newline)
		    (display "record not found ") (display inter-table) (newline)
		    #f))
	    (let ((subtable (assoc (car keys) (cdr inter-table) same-key?)))
	      (if subtable
		  (inter-lookup (cdr keys) subtable same-key?)
		  (begin 
		    (display "record not found ")(display keys)(newline)
		    #f)))))
      (inter-lookup keys local-table same-key?)) 

    (define (insert! keys value)
      (define (inter-insert keys inter-table value)
	(if (null? keys)
	    (if inter-table
		(set-cdr! inter-table value)
		#f)
	    (let ((subtable (assoc (car keys) (cdr inter-table) same-key?)))
	      (if subtable
		  (inter-insert (cdr keys) (cdr inter-table) value )
		  #f))))
      (define (inter-append keys value)
;	(display "inter-append: ") (display keys) (newline) 
	
	(if (= (length keys) 1)
	    (cons (car keys) value)
	    (append (list (car keys))  (list (inter-append (cdr keys) value)))))

      (if (inter-insert keys local-table value)
	  'ok
	  (begin
	   ; (display (inter-append keys value))(newline)
	   ; (display "---------")(newline)
	    (set! local-table (append local-table (list (inter-append keys value)))))))
    (define (display-struct)
      local-table)

    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
	    ((eq? m 'insert-proc!) insert!)
	    ((eq? m 'display-struct) display-struct)
	    
	    (else (error "Unknown operation -- TABLE" m))))
    dispatch))







In [8]:
(define (start-test-3-25)
  (define operation-table (make-table (lambda (x y) (equal? x y))))

  (define get (operation-table 'lookup-proc))

  (define put (operation-table 'insert-proc!))

  (define display-struct (operation-table 'display-struct))

  (put '(column1 row1 test ky) '11)
  (display "after put  ")
  (display (display-struct)) (newline)
  
  (put '(column2 row1) '21)
  (display "after put  ")
  (display (display-struct)) (newline)
 
 ; (display (get 'column2 'row1))
  (display "get value of column2 row1")(newline)
  (display (get '(column1 row1 test ky)))
  (newline)
  (display "after put  ")
  (display (display-struct))
  )

In [9]:
(start-test-3-25)

after put  (*table* (column1 (row1 (test (ky . 11)))))
after put  (*table* (column1 (row1 (test (ky . 11)))) (column2 (row1 . 21)))
get value of column2 row1
11
after put  (*table* (column1 (row1 (test (ky . 11)))) (column2 (row1 . 21)))